In [2]:
import numpy as np
import pandas as pd
import ta
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.linear_model import LogisticRegression


In [4]:
symbols = ['EURUSD','GBPUSD','USDCHF','USDJPY','USDCAD']

timeframe = 'H12'

df = pd.read_csv("data/"+ symbols[0]+ timeframe+ ".csv")
df = df[['Date','Open','High','Low','Close']]
df['Date'] = pd.to_datetime(df['Date'])


df['R'] =   np.log2(df['Close'] / df['Close'].shift(1)) 
df['Rshift'] = df['R'].shift(-1)

df['R2'] = np.log2(df['Close'] / df['Close'].shift(-2)) 

df['Volatilidade20'] = df['R'].rolling(3).std()

df['VolatilidadeModificada'] = [df.loc[x,'Volatilidade20'] if df.loc[x,'R'] > 0 else - df.loc[x,'Volatilidade20']  for x in df.index]

df['VolatilidadeAcumulada'] = df['VolatilidadeModificada'].rolling(5).sum()

df['Rpontos'] = df['Close'] - df['Close'].shift(1) 

df['Rpontos1'] = df['Close'].shift(-1) - df['Close'] 

df['Rpontos2'] = df['Close'].shift(-2) - df['Close'] 

df['alvo'] = [1 if x >= 0 else 0 for x in df['Rshift']]

df.dropna(inplace=True)

#df.head()
df

,Date,Open,High,Low,Close,R,Rshift,R2,Volatilidade20,VolatilidadeModificada,VolatilidadeAcumulada,Rpontos,Rpontos1,Rpontos2,alvo
7,2013-01-07 00:00:00,1.30779,1.30781,1.30165,1.30410,-0.002951,0.009285,-0.009076,0.005698,-0.005698,-0.012018,-0.00267,0.00842,0.00823,1
8,2013-01-07 12:00:00,1.30409,1.31276,1.30215,1.31252,0.009285,-0.000209,0.004955,0.006456,0.006456,0.001933,0.00842,-0.00019,-0.00450,0
9,2013-01-08 00:00:00,1.31253,1.31388,1.31025,1.31233,-0.000209,-0.004746,0.006136,0.006421,-0.006421,-0.002217,-0.00019,-0.00431,-0.00557,0
10,2013-01-08 12:00:00,1.31235,1.31309,1.30569,1.30802,-0.004746,-0.001390,0.002893,0.007160,-0.007160,-0.005909,-0.00431,-0.00126,-0.00262,0
11,2013-01-09 00:00:00,1.30804,1.30953,1.30634,1.30676,-0.001390,-0.001502,-0.003507,0.002354,-0.002354,-0.015177,-0.00126,-0.00136,0.00318,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3620,2019-12-26 00:00:00,1.10875,1.10966,1.10861,1.10918,0.000716,0.000663,-0.006087,0.001762,0.001762,0.009425,0.00055,0.00051,0.00469,1
3621,2019-12-26 12:00:00,1.10918,1.11090,1.10824,1.10969,0.000663,0.005424,-0.010286,0.000452,0.000452,0.006227,0.00051,0.00418,0.00794,1
3622,2019-12-27 00:00:00,1.10975,1.11418,1.10943,1.11387,0.005424,0.004862,-0.007841,0.002734,0.002734,0.005135,0.00418,0.00376,0.00607,1
3623,2019-12-27 12:00:00,1.11387,1.11884,1.11366,1.11763,0.004862,0.002979,-0.002940,0.002602,0.002602,0.009321,0.00376,0.00231,0.00228,1


In [4]:
indicador_bb = ta.volatility.BollingerBands(close = df['Close'], n = 20, ndev = 2, fillna = False)
indicador_rsi = ta.momentum.RSIIndicator(close = df['Close'], n = 14, fillna = False)
indicador_macd = ta.trend.MACD(close = df['Close'], n_slow = 26, n_fast = 12, n_sign = 9, fillna = False)
indicador_cci = ta.trend.CCIIndicator(high = df['High'], low = df['Low'], close = df['Close'], n = 20, c = 0.015, fillna = False)

df['bb'] = indicador_bb.bollinger_pband()
df['bbH'] = indicador_bb.bollinger_hband_indicator() 
df['bbL'] = indicador_bb.bollinger_lband_indicator() 


df['rsi'] = indicador_rsi.rsi()
df['rsiH'] = [1 if x > 70  else 0 for x in indicador_rsi.rsi()]
df['rsiL'] = [1 if x < 30  else 0 for x in indicador_rsi.rsi()]

df['macd'] =  indicador_macd.macd_diff()
df['macdH'] =  [1 if x > 0 else 0 for x in indicador_macd.macd_diff()]
df['macdL'] =  [1 if x < 0 else 0 for x in indicador_macd.macd_diff()]

df['cciH'] = [1 if x > 120  else 0 for x in indicador_cci.cci()]
df['cciL'] = [1 if x < -120  else 0 for x in indicador_cci.cci()]


df.dropna(inplace=True)
df

,Date,Open,High,Low,Close,R,Rshift,R2,Volatilidade20,VolatilidadeModificada,...,bbH,bbL,rsi,rsiH,rsiL,macd,macdH,macdL,cciH,cciL
40,2013-01-29 12:00:00,1.34261,1.34963,1.34140,1.34867,0.006443,0.007309,-0.008639,0.004496,0.004496,...,1.0,0.0,70.739498,1,0,0.000592,1,0,1,0
41,2013-01-30 00:00:00,1.34866,1.35592,1.34816,1.35552,0.007309,0.001330,0.000106,0.005110,0.005110,...,1.0,0.0,75.426068,1,0,0.001045,1,0,1,0
42,2013-01-30 12:00:00,1.35548,1.35857,1.35326,1.35677,0.001330,-0.001436,-0.002040,0.003231,0.003231,...,1.0,0.0,76.175950,1,0,0.001299,1,0,1,0
43,2013-01-31 00:00:00,1.35676,1.35836,1.35433,1.35542,-0.001436,0.003476,-0.011661,0.004470,-0.004470,...,0.0,0.0,73.565003,1,0,0.001245,1,0,1,0
44,2013-01-31 12:00:00,1.35544,1.35931,1.35409,1.35869,0.003476,0.008185,-0.005532,0.002463,0.002463,...,0.0,0.0,75.734542,1,0,0.001293,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3620,2019-12-26 00:00:00,1.10875,1.10966,1.10861,1.10918,0.000716,0.000663,-0.006087,0.001762,0.001762,...,0.0,0.0,47.918709,0,0,-0.000762,0,1,0,0
3621,2019-12-26 12:00:00,1.10918,1.11090,1.10824,1.10969,0.000663,0.005424,-0.010286,0.000452,0.000452,...,0.0,0.0,49.241160,0,0,-0.000625,0,1,0,0
3622,2019-12-27 00:00:00,1.10975,1.11418,1.10943,1.11387,0.005424,0.004862,-0.007841,0.002734,0.002734,...,0.0,0.0,58.534564,0,0,-0.000245,0,1,0,0
3623,2019-12-27 12:00:00,1.11387,1.11884,1.11366,1.11763,0.004862,0.002979,-0.002940,0.002602,0.002602,...,0.0,0.0,64.781050,0,0,0.000242,1,0,1,0


In [28]:
v = 'bb'
v ='rsi'
#v ='VolatilidadeModificada'
r = 'Rpontos2'

a = np.arange(0,1.05, 0.05)
b = np.quantile(df[v],a)

Tr = df[df['Date'] < '2019-01-01']

array = []

for i in range(1, len(b)):
    array.append([b[i-1], b[i] , Tr[(Tr[v] > b[i-1])  & (Tr[v] < b[i])][r].sum()])

data = pd.DataFrame(array ,columns = ['a','b','retorno'])

data.sort_values('retorno')


,a,b,retorno
16,59.411471,61.868239,-0.17043
2,33.141372,35.717060,-0.12084
15,57.145100,59.411471,-0.10136
18,64.794151,68.919224,-0.07709
19,68.919224,83.715267,-0.07696
3,35.717060,38.032341,-0.06409
9,47.517541,49.183052,-0.06253
13,53.701630,55.402850,-0.05557
0,14.265355,29.019849,-0.04424
4,38.032341,40.061195,-0.04011


In [39]:
Ts = df[df['Date'] >= '2019-01-01']
l = 1
a = data.loc[l,'a']
b = data.loc[l,'b']

Ts[(Ts['bbH'] == 1) & (Ts[v] > a)  & (Ts[v] < b)  & (Ts['bbL'] == 1)]



,Date,Open,High,Low,Close,R,Rshift,R2,Volatilidade20,VolatilidadeModificada,...,bbH,bbL,rsi,rsiH,rsiL,macd,macdH,macdL,cciH,cciL


In [ ]:
variaveis = ['bbH','bbL', 'rsiH', 'rsiL', 'macd', 'cciH', 'cciL']

X_treino = df[df['Date'] < '2019-01-01'][variaveis]
y_treino = df[df['Date'] < '2019-01-01']['alvo']

X_teste = df[df['Date'] >= '2019-01-01'][variaveis]
y_teste = df[df['Date'] >='2019-01-01']['alvo']

In [ ]:
X = df[variaveis]
y = df['alvo']
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.6, random_state=1)

In [ ]:
modelo = LogisticRegression(random_state=1)

modelo.fit(X_treino,y_treino)

p = modelo.predict(X_teste)

np.mean(p==y_teste)